In [3]:
import os
import sys
import os
from collections import defaultdict

src_path = os.path.abspath(os.path.join(os.getcwd(), ".."))
if src_path not in sys.path:
    sys.path.append(src_path)
DATA_BASEPATH = os.path.abspath(os.path.join(os.getcwd(), "..","../Data"))

from Util.posttree import PostTree
from eval import EvalSimilarity

In [5]:
POST_IDS = ['18w4y9z', '1au5njg', '1atcjtj'] 
TEST_TYPE = 'instruct'  # [few_shot, zero_shot, instruct]

for POST_ID in POST_IDS:
    POST_PATH = os.path.abspath(os.path.join(DATA_BASEPATH, 'arcticshift/processed/r_books_posts.csv'))
    ORG_COMMENTPATH = os.path.abspath(os.path.join(DATA_BASEPATH, 'arcticshift/cleaned/r_books_comments_cleaned_and_pruned.csv'))

    # reconstruct original tree
    post_tree = PostTree(
        POST_PATH,
        POST_ID
    )
    post_tree.create_comment_tree(ORG_COMMENTPATH)

    all_levels = defaultdict(float)
    all_counts = defaultdict(int)

    post_folder = os.path.join(DATA_BASEPATH, "eval_results", TEST_TYPE, post_tree.post_id)
    os.makedirs(post_folder, exist_ok=True)
    averages_file = os.path.join(post_folder, "final_averages_results.txt")

    if os.path.exists(averages_file):
        os.remove(averages_file)

    for i in range(10):
        GEN_COMMENTPATH = os.path.abspath(os.path.join(DATA_BASEPATH, f'posttrees/{TEST_TYPE}/{POST_ID}/copy_{i}.json'))

        gen_posttree = PostTree.load_from_json(GEN_COMMENTPATH)

        levels = defaultdict(float)
        counts = defaultdict(int)

        similarity = EvalSimilarity(post_tree, gen_posttree)

        bfs_file = os.path.join(post_folder, f"bfs_results_copy_{i}.txt")
        if os.path.exists(bfs_file):
            os.remove(bfs_file)

        seen = set()

        similarity_tree = similarity.compare_comments()

        for root_node in similarity_tree:
            if root_node.comment_id not in seen:
                similarity.bfs(root_node, levels, counts, bfs_file)
                seen.add(root_node.comment_id)

        # accumulate levels/counts into the all_levels and all_counts
        for depth in levels.keys():
            all_levels[depth] += levels[depth]
            all_counts[depth] += counts[depth]

    with open(averages_file, "w") as file:
        file.write("Final Average Similarity Scores by Depth (across all 10 generated trees):\n")
        print("\nFinal Average Similarity Scores by Depth (across all 10 generated trees):")
        for depth in sorted(all_levels.keys()):
            if all_counts[depth] > 0:
                average = all_levels[depth] / all_counts[depth]
                line = f"Depth {depth}: Average Similarity Score = {average:.2f}"
                print(line)
                file.write(line + "\n")
            else:
                # If for some reason no counts, just skip or note it
                line = f"Depth {depth}: No data."
                print(line)
                file.write(line + "\n")


['Eragon? It tried to adapt the story but it utterly failed.', "The movie version of A Wrinkle in Time was extremely disappointing. The book is a classic for a reason, with its imaginative world-building, compelling characters, and intricate plot. The movie, on the other hand, seemed to focus more on flashy visuals and musical numbers, losing much of the original's charm and depth. The cast, while great actors, also seemed ill-suited for their roles, with some struggling to capture the unique qualities of the characters in the book. Overall, I would say that A Wrinkle in Time is a prime example of a beloved book being poorly adapted into a movie. :("]
['The Golden Compass', '1. The movie adaptation of Stephen King\'s "Shawshank Redemption" was very disappointing to me. While the book was able to build tension and suspense, the movie seemed to miss the mark. It was too focused on the escape plot, and missed the emotional depth that the book had.\n        \u200b\n        2. The movie ada